In [1]:
import numpy as np
import pandas as pd

In [2]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder

In [5]:
df = pd.read_csv('covid_toy.csv')
df.sample(5)

,age,gender,fever,cough,city,has_covid
45,72,Male,99.0,Mild,Bangalore,No
27,33,Female,102.0,Strong,Delhi,No
83,17,Female,104.0,Mild,Kolkata,No
38,49,Female,101.0,Mild,Delhi,Yes
9,64,Female,101.0,Mild,Delhi,No


In [7]:
df.isnull().sum()

age           0
gender        0
fever        10
cough         0
city          0
has_covid     0
dtype: int64

In [10]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(df.drop(columns=['has_covid']),
                                                 df['has_covid'],
                                                 test_size=0.2)

In [11]:
X_train

,age,gender,fever,cough,city
3,31,Female,98.0,Mild,Kolkata
43,22,Female,99.0,Mild,Bangalore
26,19,Female,100.0,Mild,Kolkata
90,59,Female,99.0,Strong,Delhi
65,69,Female,102.0,Mild,Bangalore
...,...,...,...,...,...
73,34,Male,98.0,Strong,Kolkata
0,60,Male,103.0,Mild,Kolkata
7,20,Female,NaN,Strong,Mumbai
89,46,Male,103.0,Strong,Bangalore


In [14]:
# adding simple imputer to fever col
si = SimpleImputer()
X_train_fever = si.fit_transform(X_train[['fever']])

# also the test data
X_test_fever = si.fit_transform(X_test[['fever']])

X_test_fever.shape

(20, 1)

In [17]:
# Ordinalencoding -> cough
oe = OrdinalEncoder(categories=[['Mild','Strong']])

X_train_cough = oe.fit_transform(X_train[['cough']])
X_test_cough = oe.fit_transform(X_test[['cough']])

X_train_cough.shape

(80, 1)

In [19]:
# OneHotEncoding -> gender,city
ohe = OneHotEncoder(drop='first',sparse_output=False)
X_train_gender_city = ohe.fit_transform(X_train[['gender','city']])

# also the test data
X_test_gender_city = ohe.fit_transform(X_test[['gender','city']])

X_train_gender_city.shape

(80, 4)

In [26]:
# Extracting Age
X_train_age = X_train.drop(columns=['gender','fever','cough','city']).values

# also the test data
X_test_age = X_test.drop(columns=['gender','fever','cough','city']).values

X_train_age.shape

(80, 1)

In [28]:
X_train_transformed = np.concatenate((X_train_age,X_train_fever,X_train_gender_city,X_train_cough),axis=1)

In [ ]:
X_train_transformed

array([[ 31.        ,  98.        ,   0.        ,   0.        ,
          1.        ,   0.        ,   0.        ],
       [ 22.        ,  99.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ],
       [ 19.        , 100.        ,   0.        ,   0.        ,
          1.        ,   0.        ,   0.        ],
       [ 59.        ,  99.        ,   0.        ,   1.        ,
          0.        ,   0.        ,   1.        ],
       [ 69.        , 102.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ],
       [ 40.        ,  98.        ,   0.        ,   1.        ,
          0.        ,   0.        ,   1.        ],
       [ 81.        ,  98.        ,   0.        ,   0.        ,
          0.        ,   1.        ,   1.        ],
       [ 26.        ,  98.        ,   0.        ,   0.        ,
          1.        ,   0.        ,   0.        ],
       [ 65.        ,  99.        ,   1.        ,   1.        ,
          0.    

In [32]:
X_test_transformed = np.concatenate((X_test_age,X_test_fever,X_test_gender_city,X_test_cough),axis=1)
X_test_transformed.shape

(20, 7)

### ColumnTransformer

In [33]:
from sklearn.compose import ColumnTransformer

In [45]:
transformer = ColumnTransformer(transformers=[
    ('tnf1',SimpleImputer(),['fever']),
    ('tnf2',OrdinalEncoder(categories=[['Mild','Strong']]),['cough']),
    ('tnf3',OneHotEncoder(drop='first',sparse_output=False),['gender','city'])
],remainder='passthrough')

In [46]:
transformer.fit_transform(X_train).shape

(80, 7)

In [47]:
transformer.fit_transform(X_test).shape

(20, 7)